In [1]:
import numpy as np
import pandas as pd

In [2]:
def donchian(price, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(price.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    df['buy'] = np.where(df.close > df.upr, 1, 0)
    df['sell'] = np.where(df.close < df.mid, 1, 0)
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'buy'] == 1 and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'sell'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
    df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)
    return df

def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df


def remainder_zero(series, divisor):
    '''Increment number until remainder is zero.'''
    def increment_dividend(dividend, divisor):
        '''Increment dividend while remainder does not equal zero.'''
        while dividend % divisor != 0:
            dividend += 1
        return dividend
    series = series.map(lambda x: increment_dividend(x, divisor))
    return series

In [3]:
# Input variables.
exchange = 'LSE'
tidm = 'HSV'
period = pd.Series({'1': 48, '2': 24, '3': 12, '4': 6})  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
price = weekly(exchange, tidm)

In [5]:
# System 1 Donchian channel.
dc1 = donchian(price, period.loc['1'])
dc1

,open,high,low,close,upr,lwr,mid,buy,sell,state,entry,exit
date,,,,,,,,,,,,
1994-01-07,NaN,NaN,NaN,0.33234,NaN,NaN,NaN,0,0,0,0,0
1994-01-14,NaN,NaN,NaN,0.33385,NaN,NaN,NaN,0,0,0,0,0
1994-01-21,NaN,NaN,NaN,0.33815,NaN,NaN,NaN,0,0,0,0,0
1994-01-28,NaN,NaN,NaN,0.34031,NaN,NaN,NaN,0,0,0,0,0
1994-02-04,NaN,NaN,NaN,0.34095,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13,9.925,11.25,9.560,10.69000,10.24,5.79,8.015,1,0,1,1,0
2022-05-20,10.630,11.75,10.310,11.64000,11.25,5.79,8.520,1,0,1,0,0
2022-05-27,11.635,11.68,11.585,11.65000,11.75,5.79,8.770,0,0,1,0,0


In [7]:
# # Position Size.
# db = pd.DataFrame(df[df.entry == 1])
# db['volatility'] = abs((db.mid - db.close) / db.close)
# db['risk_amt'] = (position_size * risk_pct) / db.volatility
# db['shares'] = (db.risk_amt / db.close).astype('int')
# (
#     db.style
#         .format_index(lambda s: s.strftime("%Y-%m-%d"))
#         .format({
#             'open': '{:.3f}',
#             'high': '{:.3f}',
#             'low': '{:.3f}',
#             'close': '{:.3f}',
#             'upr': '{:.3f}',
#             'lwr': '{:.3f}',
#             'mid': '{:.3f}',
#             'volatility': '{:.1%}',
#             'risk_amt': '{:,.2f}',
#         })
# )